<a href="https://colab.research.google.com/github/alex-jk/painting-lora-finetune/blob/main/combine_photos_image_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Combine photos and generate new similar image*